In [11]:
import torch
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import numpy as np

# Load the fine-tuned model
model_path = "checkpoints/model_epoch_1.pt"  # Update with the correct path
model_state_dict = torch.load(model_path)
model = RobertaForSequenceClassification.from_pretrained("roberta-base")
model.load_state_dict(model_state_dict)


# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Define the input review
input_review = "The XYZ mobile phone offers exceptional performance and a stunning display, making it a top choice for tech enthusiasts."

# Tokenize the input
input_ids = tokenizer(input_review, return_tensors="pt", padding=True, truncation=True)
input_ids = input_ids["input_ids"]

# Pass the input through the model
with torch.no_grad():
    output = model(input_ids)

# Get the predicted label
predicted_label = int(np.argmax(output.logits))

# Interpret the label (0 for ChatGPT, 1 for human)
if predicted_label == 0:
    print("ChatGPT")
else:
    print("Human")

model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ChatGPT


NameError: name 'device' is not defined

In [9]:

# Load the labeled dataset
df = pd.read_csv("labeled-dataset.csv")


# Split the dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


# Define the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


# Tokenization and DataLoader
max_length = 128  # Adjust as needed
train_dataset = CustomDataset(train_df, tokenizer, max_length)
val_dataset = CustomDataset(val_df, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


In [14]:
import torch
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Load the labeled dataset
df = pd.read_csv("labeled-dataset.csv")

# Split the dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Define the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Define a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        review = str(self.data.iloc[index]['text'])
        label = int(self.data.iloc[index]['label'])  # Assuming labels are integers
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Tokenization and DataLoader
max_length = 128  # Adjust as needed
train_dataset = CustomDataset(train_df, tokenizer, max_length)
val_dataset = CustomDataset(val_df, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Initialize the RoBERTa model
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
model.eval()
val_loss = 0
all_labels = []
all_preds = []
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        val_loss += loss.item()
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds)

accuracy = accuracy_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=["ChatGPT", "Human"])
print(f"Validation Loss: {val_loss / len(val_loader)}")
print(f"Accuracy: {accuracy}")
print(report)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Validation Loss: 0.6938140824729321
Accuracy: 0.5204460966542751
              precision    recall  f1-score   support

     ChatGPT       0.52      1.00      0.68       420
       Human       0.00      0.00      0.00       387

    accuracy                           0.52       807
   macro avg       0.26      0.50      0.34       807
weighted avg       0.27      0.52      0.36       807



c:\Users\Fusion\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Fusion\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Fusion\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera